In [140]:
import os
import tempfile
import urllib
import pandas as pd
from hmmlearn.base import ConvergenceMonitor
from hmmlearn import hmm

import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import ml_metadata as mlmd
print('MLMD version: {}'.format(mlmd.__version__))

TFX version: 1.2.0
MLMD version: 1.2.0


In [141]:
model = hmm.GMMHMM(n_components=2, tol=5, verbose=True)

In [142]:
eurDf = pd.read_csv('/Users/marae/source/repos/Machine-Learning-Engineering/MLOps Coursera/MLlifeProd/TensorFlow Data Transform Tutorial/EURUSD_D1.csv')
eurDf.drop(columns='Date',inplace=True)
eurDf.rename(columns={'Open':'open','Low':'low','High':'high','Close':'close'},inplace=True)

eurModel = model.fit(eurDf)

C:\Users\marae\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
         1      -52930.3540             +nan
         2      -46106.3841       +6823.9699
         3      -42485.8533       +3620.5309
         4      -40576.0176       +1909.8357
         5      -39751.7112        +824.3063
         6      -39563.6301        +188.0811
         7      -39520.9936         +42.6365
         8      -39517.1873          +3.8063


In [143]:
decode = eurModel.decode(eurDf)
sampleScores = eurModel.score_samples(eurDf)
scoresDf = pd.DataFrame(sampleScores[1])
eurDf['Regime'] = scoresDf[0].astype('string')

In [144]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
eurDf.to_csv(_data_root + '\\EURUSD_D1Markov.csv',index=False)

In [145]:
interactive_context = InteractiveContext()
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\CsvExampleGen\\examples\\1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:260175,xor_checksum:1630120275,sum_checksum:1630120275"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [146]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\StatisticsGen\\statistics\\2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [147]:
infer_schema = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
interactive_context.run(infer_schema)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\SchemaGen\\schema\\3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [148]:
# Define the module file for the Trainer component
trainer_module_file = 'eurUsdTrainer.py'

In [149]:
%%writefile {trainer_module_file}

# Define the training algorithm for the Trainer module file
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

# Features used for classification - culmen length and depth, flipper length,
# body mass, and species.

_LABEL_KEY = 'Regime'

_FEATURE_KEYS = [
    'high', 'low', 'open', 'Volume','close'
]


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema, batch_size: int) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY), schema).repeat()


def _build_keras_model():
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation='relu')(d)
  d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])
  return model


def run_fn(fn_args: tfx.components.FnArgs):
  schema = schema_pb2.Schema()
  tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema)
  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=10)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=10)
  model = _build_keras_model()
  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / 20),
      steps_per_epoch=20,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting eurUsdTrainer.py


In [150]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module_file),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=50))
interactive_context.run(trainer)

Epoch 1/5
20/20 [==============================] - ETA: 8s - loss: 21168.2461 - sparse_categorical_accuracy: 0.400 - 1s 14ms/step - loss: 2689.6455 - sparse_categorical_accuracy: 0.2550 - val_loss: 1264.1918 - val_sparse_categorical_accuracy: 0.0880
Epoch 2/5
20/20 [==============================] - ETA: 0s - loss: 704.9036 - sparse_categorical_accuracy: 0.0000e+0 - 0s 8ms/step - loss: 1493.7979 - sparse_categorical_accuracy: 0.1550 - val_loss: 211.3783 - val_sparse_categorical_accuracy: 0.0860
Epoch 3/5
20/20 [==============================] - ETA: 0s - loss: 199.9038 - sparse_categorical_accuracy: 0.100 - 0s 8ms/step - loss: 974.8555 - sparse_categorical_accuracy: 0.2200 - val_loss: 663.4934 - val_sparse_categorical_accuracy: 0.4200
Epoch 4/5
20/20 [==============================] - ETA: 0s - loss: 343.2339 - sparse_categorical_accuracy: 0.300 - 0s 9ms/step - loss: 538.7542 - sparse_categorical_accuracy: 0.1950 - val_loss: 1075.5186 - val_sparse_categorical_accuracy: 0.1080
Epoch 5/5

INFO:tensorflow:Assets written to: C:\Users\marae\AppData\Local\Temp\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\Trainer\model\4\Format-Serving\assets


INFO:tensorflow:Assets written to: C:\Users\marae\AppData\Local\Temp\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\Trainer\model\4\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 4
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\Trainer\\model\\4"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 5
        type_id: 21
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\Trainer\\model_run\\4"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [151]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/eurUsd_classification')

In [152]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Regime', signature_name='serving_default')
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.6})))
        ])
    ],
    slicing_specs=[tfma.SlicingSpec()])

In [153]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    schema=infer_schema.outputs['schema'],
    eval_config=eval_config)
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 5
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 6
        type_id: 23
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\Evaluator\\evaluation\\5"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 7
        type_id: 24
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\Evaluator\\blessing\\5"
        custom_properties {
          key: "blessed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\Trainer\\model\\4"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 4
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [154]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 6
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 8
        type_id: 26
        uri: "C:\\Users\\marae\\AppData\\Local\\Temp\\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\\Pusher\\pushed_model\\6"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [155]:
connection_config = interactive_context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

In [156]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

In [157]:
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

In [158]:
def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

In [159]:
display_types(store.get_artifact_types())

,id,name
0,14,Examples
1,16,ExampleStatistics
2,18,Schema
3,20,Model
4,21,ModelRun
5,23,ModelEvaluation
6,24,ModelBlessing
7,26,PushedModel


In [160]:
pushed_models = store.get_artifacts_by_type("PushedModel")
display_artifacts(store, pushed_models)

,artifact id,type,uri
0,8,PushedModel,./Pusher\pushed_model\6


In [161]:
pushed_model = pushed_models[-1]
display_properties(store, pushed_model)

,property,value
0,tfx_version,1.2.0
1,name,pushed_model
2,producer_component,Pusher
3,state,published
4,pushed,0


In [162]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

In [163]:
parent_artifacts = get_one_hop_parent_artifacts(store, [pushed_model])
display_artifacts(store, parent_artifacts)

,artifact id,type,uri
0,4,Model,./Trainer\model\4
1,7,ModelBlessing,./Evaluator\blessing\5


In [164]:
exported_model = parent_artifacts[0]
display_properties(store, exported_model)

,property,value
0,state,published
1,name,model
2,tfx_version,1.2.0
3,producer_component,Trainer


In [165]:
model_parents = get_one_hop_parent_artifacts(store, [exported_model])
display_artifacts(store, model_parents)

,artifact id,type,uri
0,1,Examples,./CsvExampleGen\examples\1
1,3,Schema,./SchemaGen\schema\3


In [166]:
used_data = model_parents[0]
display_properties(store, used_data)

,property,value
0,split_names,"[""train"", ""eval""]"
1,file_format,tfrecords_gzip
2,input_fingerprint,"split:single_split,num_files:1,total_bytes:260175,xor_checksum:1630120275,sum_checksum:1630120275"
3,payload_format,FORMAT_TF_EXAMPLE
4,tfx_version,1.2.0
5,state,published
6,span,0


In [167]:
display_types(store.get_execution_types())

,id,name
0,12,tfx.components.example_gen.csv_example_gen.component.CsvExampleGen
1,15,tfx.components.statistics_gen.component.StatisticsGen
2,17,tfx.components.schema_gen.component.SchemaGen
3,19,tfx.components.trainer.component.Trainer
4,22,tfx.components.evaluator.component.Evaluator
5,25,tfx.components.pusher.component.Pusher


In [168]:
def find_producer_execution(store, artifact):
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids([artifact.id])
      if event.type == mlmd.proto.Event.OUTPUT)
  return store.get_executions_by_id(executions_ids)[0]

trainer = find_producer_execution(store, exported_model)
display_properties(store, trainer)

,property,value
0,custom_config,null
1,trainer_fn,None
2,run_fn,None
3,state,complete
4,component_id,Trainer
5,eval_args,"{\n ""num_steps"": 50\n}"
6,pipeline_name,interactive-2021-08-27T23_11_15.421452
7,train_args,"{\n ""num_steps"": 100\n}"
8,module_path,eurUsdTrainer@C:\Users\marae\AppData\Local\Temp\tfx-interactive-2021-08-27T23_11_15.421452-lvlljevn\_wheels\tfx_user_code_Trainer-0.0+1442405bb5a7544c9036747f263f25f2d05b5112c4268735695db31f5a245d98-py3-none-any.whl
9,run_id,2021-08-27T23:11:22.072975


In [169]:
serving_Data_Root = tempfile.mkdtemp(prefix='tfx-data')
servingData = pd.read_csv('/users/marae/source/repos/Machine-Learning-Engineering/MLOps Coursera/MLlifeProd/TensorFlow Metadata Tutorial/FX_EURUSD, 5.csv')
servingData.to_csv(serving_Data_Root + '\\EURUSD_New.csv',index=False)

serving_context = InteractiveContext()
serving_example_gen = tfx.components.CsvExampleGen(input_base=serving_Data_Root)

serving_statistics_gen = tfx.components.StatisticsGen(
    examples=serving_example_gen.outputs['examples'])

serving_schema = tfx.components.SchemaGen(
    statistics=serving_statistics_gen.outputs['statistics'], infer_feature_shape=True)

In [170]:
statsVal = tfdv.generate_statistics_from_csv(data_location=_data_root + '\\EURUSD_D1Markov.csv')
schema = tfdv.infer_schema(statistics=statsVal)
stats_options = tfdv.StatsOptions(schema=schema,infer_type_from_schema=True)
eval_stats = tfdv.generate_statistics_from_csv(data_location=serving_Data_Root + '\\EURUSD_New.csv', stats_options=stats_options)

serving_anomalies = tfdv.validate_statistics(eval_stats,schema)
tfdv.display_anomalies(serving_anomalies)

C:\Users\marae\anaconda3\lib\site-packages\tensorflow_data_validation\utils\display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'Regime',Column dropped,Column is completely missing


In [172]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

tfdv.get_feature(schema,'Regime').not_in_environment.append('SERVING')

serving_anomalies = tfdv.validate_statistics(eval_stats, schema, environment='SERVING')
tfdv.display_anomalies(serving_anomalies)

C:\Users\marae\anaconda3\lib\site-packages\tensorflow_data_validation\utils\display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)
